# Notebook to compute Overall Return Period

This notebook computes the return period for the action and observational triggers.

In [1]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [7]:
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from src.datasources import codab, rsmc
from src.constants import *
from src import utils

In [8]:
adm1 = codab.load_codab(admin_level=1)

In [9]:
df_rsmc = rsmc.load_historical_forecast_distances()
distance_cols = [x for x in df_rsmc.columns if "_distance_km" in x]
df_rsmc["any_distance_km"] = df_rsmc[distance_cols].min(axis=1)

## Readiness


In [12]:
storms_readiness = ["IDAI", "FREDDY", "GOMBE", "ELOISE", "CHALANE"]

In [13]:
set(storms_readiness)

{'CHALANE', 'ELOISE', 'FREDDY', 'GOMBE', 'IDAI'}

In [17]:
readiness_yr_len = 14
df_readiness_true = len(storms_readiness)
print(
    f"The readiness trigger is met every {round(readiness_yr_len / df_readiness_true, 1)} years."
)

The readiness trigger is met every 2.8 years.


In [18]:
readiness_rate = df_readiness_true / readiness_yr_len
readiness_rate

0.35714285714285715

## Action

In [19]:
action_yr_len = 14

In [27]:
action = set(["IDAI", "FREDDY", "GOMBE", "ELOISE"])
print(
    f"The action trigger is met every {round(action_yr_len / len(action), 1)} years."
)

The action trigger is met every 3.5 years.


In [28]:
action_rate = len(action) / action_yr_len
action_rate

0.2857142857142857

## Observational: Wind Speed

In [97]:
obs_ws_yr_len = 25
obs_ws = set(
    [
        "IDAI",
        "DINEO",
        "ELOISE",
        "KENNETH",
        "JOKWE",
        "FAVIO",
        "JAPHET",
        "HUDAH",
        "ELINE:LEONE",
    ]
)
print(
    f"The observational wind speed trigger is met every {round(obs_ws_yr_len / len(obs_ws), 1)} years."
)

The observational wind speed trigger is met every 2.8 years.


In [98]:
obs_ws_rate = len(obs_ws) / obs_ws_yr_len
obs_ws_rate

0.36

## Observational: Rainfall

In [99]:
obs_rain_yr_len = 23

In [100]:
# From notebook 03.2_historical_rainfall
obs_rain = set(
    ["IDAI", "FREDDY", "GOMBE", "KENNETH", "ANA", "DESMOND", "JAPHET"]
)
print(
    f"The observational rainfall trigger is met every {round(obs_rain_yr_len / len(obs_rain), 1)} years."
)

The observational rainfall trigger is met every 3.3 years.


In [101]:
obs_rain_rate = len(obs_rain) / obs_rain_yr_len
obs_rain_rate

0.30434782608695654

## Combined Observational

### Combined probability (at least one activates)

#### Wind Speed and Rainfall

We have to account for the 2 cyclones for which we do not have rainfall data. Both Hudah and Eline Leone were associated with significant rainfall. To account for at least one reaching the threshold, we should adjust the intersection.

In [102]:
common_cyclones = obs_ws.intersection(obs_rain)
# common_cyclones.update({"HUDAH", "ELINE:LEONE"})
total_cyclones = obs_ws.union(obs_rain)

probability_intersection = (len(common_cyclones) + 1) / len(total_cyclones)

print(f"Common Cyclones: {common_cyclones}")
print(f"Probability of Intersection: {probability_intersection:.2f}")
print(
    f"Return Period: 1-in-{round(1 / (obs_ws_rate + obs_rain_rate - probability_intersection), 1)} yr"
)

Common Cyclones: {'IDAI', 'KENNETH', 'JAPHET'}
Probability of Intersection: 0.31
Return Period: 1-in-2.8 yr
